# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest 
pulling dde5aa3fc5ff... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 2.0 GB                         
pulling 966de95ca8a6... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.4 KB                         
pulling fcc5a6bec9da... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 7.7 KB                         
pulling a70ff7e570d9... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 6.0 KB                         
pulling 56bb8bd477a5... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   96 B                         
pulling 34bb5ab01051... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  561 B                         
verifying sha256 digest 
writing manifest â ‹ pulling manifest 
pulling dde5aa3fc5ff... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 2.0 GB      

In [6]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can be used to generate high-quality content such as blog posts, social media posts, product descriptions, and even entire articles. This can help businesses save time and money on content creation.
2. **Design and Visual Content**: Generative AI can be used to create custom designs for products, packaging, branding materials, and more. It can also generate images, logos, and graphics with unprecedented speed and accuracy.
3. **Marketing Automation**: Generative AI can be used to automate marketing tasks such as lead generation, email marketing, and social media posting. This can help businesses save time and resources on manual marketing efforts.
4. **Personalized Customer Experience**: Generative AI can be used to generate personalized content for customers based on their preferences, behavior, and demographics. This can help businesses improve customer engag

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [7]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI (also known as Generative Models or Generative Adversarial Networks, GANs) has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can be used to generate high-quality content such as:
 * Articles and blog posts
 * Social media posts
 * Product descriptions
 * Video scripts
 * Music and audio tracks
2. **Image and Video Generation**: Generative AI can be used to create realistic images and videos for:
 * Product photography
 * Advertising campaigns
 * Virtual try-on experiences
 * Storytelling in e-learning platforms
3. **Chatbots and Customer Service**: Generative AI-powered chatbots can:
 * Provide 24/7 customer support
 * Offer personalized recommendations
 * Help with transactions and order fulfillment
4. **Data Analysis and Visualization**: Generative AI can be used to analyze and visualize complex data sets, such as:
 * Financial market analysis
 * Weather forecasting
 * Medical diagnosis
5. **P

## Alternative approach - using OpenAI python library to connect to Ollama

In [9]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation:** Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and more. This can help businesses save time and resources on content creation while maintaining consistency and quality.
2. **Product Design:** AI-powered generative designs can create prototypes for new products, allowing designers to quickly test and refine ideas without the need for expensive prototyping and testing processes.
3. **Marketing Copywriting:** Generative AI can also be used to generate high-quality marketing copy, such as email subject lines, product pitches, and advertising scripts, that are tailored to specific target audiences.
4. **Social Media Management:** AI-powered generative tools can help businesses create engaging social media content, such as videos, images, and posts, that resonate with their audience.
5. **Influe

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [9]:
!ollama pull deepseek-r1:1.5b

Der Befehl "ollama" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [10]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

<think>
Okay, so I need to define the key concepts behind Large Language Models (LLMs), focusing on something called "neural networks," "attention," and "transformers." Hmm. Let me think about this step by step.

Starting with neural networks... I remember that LLMs are a type of AI model based on these neural network technologies. I think they process information through layers of nodes, or neurons, similar to the way our brains work for recognizing patterns. Each layer processes data sequentially until it's processed in parallel. But wait, what makes them " intelligent"? I recall that each neuron has some memory, or context, and can learn from examples. Plus, weights are adjusted during training using backpropagation. This self-supervised learning aspect is probably crucial because without that, traditional neural networks just pass through the data.

Next, attention. If this is talking about Transformers as well, I remember "attention" is a key mechanism developed by researchers lik

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [17]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')


headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

def summarize(url, model):
    website = Website(url)
    if model == "llama3.2":
        response = ollama.chat(
            model="llama3.2",
            messages = messages_for(website)
        )
        return response['message']['content']
        
    if model == "openai":
        response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
        )
        return response.choices[0].message.content

    if model == "deepseek":
        response = ollama_via_openai.chat.completions.create(
        model="deepseek-r1:1.5b",
        messages = messages_for(website)
        )
        return response.choices[0].message.content
        
    else:
        print(f"Model {model} is not currently supported for summarisation. Please choose one of 'openai', 'llama3.2' or 'deepseek'.")
              
def display_summary(url, model):
    summary = summarize(url, model)
    display(Markdown(summary))


In [18]:
display_summary("https://edwarddonner.com", "llama3.2")

# Website Summary
=====================================

The website "Home - Edward Donner" appears to be a personal blog or profile of its creator, Ed. The content is mostly about his interests and work in the field of Artificial Intelligence (AI), particularly Large Language Models (LLMs).

## Key Points

* Ed is the co-founder and CTO of Nebula.io, an AI startup that applies LLMs to help people discover their potential.
* He has previously founded and led another AI startup, untapt, which was acquired in 2021.
* The website features news and announcements about Ed's projects and endeavors, including:
	+ LLM Workshop – Hands-on with Agents (January 23, 2025)
	+ Welcome, SuperDataScientists! (November 13, 2024)
	+ Mastering AI and LLM Engineering – Resources (October 16, 2024)
* Ed is active on social media platforms such as LinkedIn, Twitter, Facebook, and has a newsletter subscription.

## Tone and Style

The website has a casual and conversational tone, with Ed sharing his thoughts on various topics related to AI and technology. The content appears to be geared towards enthusiasts and professionals in the field of AI and machine learning.